In [1]:
#hide
#default_exp vis.gen

# Visualisation Generation

<br>

### Imports

In [2]:
#exports
import json
import pandas as pd

import importlib
from tqdm import tqdm

import matplotlib.pyplot as plt

In [3]:
from IPython.display import JSON

In [4]:
#exports
def rgb_2_plt_tuple(r, g, b):
    """converts a standard rgb set from a 0-255 range to 0-1"""
    
    plt_tuple = tuple([x/255 for x in (r, g, b)])
    
    return plt_tuple

In [6]:
vis_configs = [
    {
        'cron': '* * 0 * *', # midnight every monday
        'function': 'ElexonDataPortal.vis.ei.generate_GB_decarb_progess',
        'kwargs': {
            'dt_col': 'local_datetime',
            'dt_tz': 'Europe/London',
            'url': 'https://api.github.com/repos/AyrtonB/Electric-Insights/git/trees/master?recursive=1',
            'raw_file_prefix': 'https://raw.githubusercontent.com/AyrtonB/Electric-Insights/master/',
            'docs_dir': 'docs',
            'update_time': pd.Timestamp.now().round('5min').strftime('%Y-%m-%d %H:%M'),
            'dpi': 250,
            'freq': '7D',
            'use_preloaded_ei_df': True,
            'fuel_colour_dict': {
                'Imports & Storage' : rgb_2_plt_tuple(121,68,149), 
                'nuclear' : rgb_2_plt_tuple(77,157,87), 
                'biomass' : rgb_2_plt_tuple(168,125,81), 
                'gas' : rgb_2_plt_tuple(254,156,66), 
                'coal' : rgb_2_plt_tuple(122,122,122), 
                'hydro' : rgb_2_plt_tuple(50,120,196), 
                'wind' : rgb_2_plt_tuple(72,194,227), 
                'solar' : rgb_2_plt_tuple(255,219,65),
            }
        }
    },
    {
        'cron': '* * 0 * *', # midnight every monday
        'function': 'ElexonDataPortal.vis.ei.generate_moe',
        'kwargs': {
            'dt_col': 'local_datetime',
            'dt_tz': 'Europe/London',
            'url': 'https://api.github.com/repos/AyrtonB/Electric-Insights/git/trees/master?recursive=1',
            'raw_file_prefix': 'https://raw.githubusercontent.com/AyrtonB/Electric-Insights/master/',
            'reg_dates_start': '2010-01-01',
            'reg_dates_end': None,
            'reg_dates_freq': '13W',
            'num_fits': 15, 
            'x_pred': None,
            'dt_idx': None,
            'dpi': 250, 
            'use_preloaded_ei_df': True,
            'img_name': 'moe_surface',
            'docs_dir': 'docs',
        }
    }
]

JSON(vis_configs)

<IPython.core.display.JSON object>

In [7]:
save_vis_configs = False

if save_vis_configs == True:
    with open('../data/vis_configs.json', 'w') as f:
        json.dump(vis_configs, f)

In [8]:
#exports
def get_vis_func(func_path):
    *lib_path, func_name = func_path.split('.')
    lib_obj = importlib.import_module('.'.join(lib_path))
    
    func = getattr(lib_obj, func_name)

    return func

def get_vis_md_text(vis_config, docs_dir=None):
    func_path = vis_config['function']
    kwargs = vis_config['kwargs']

    if (docs_dir is not None) and ('docs_dir' in kwargs.keys()):
        kwargs['docs_dir'] = docs_dir

    vis_func = get_vis_func(func_path)
    vis_md_text = vis_func(**kwargs)
    plt.close()
    
    return vis_md_text

In [9]:
docs_dir = '../docs'
all_vis_md_texts = []

for vis_config in vis_configs:
    vis_md_text = get_vis_md_text(vis_config, docs_dir=docs_dir)
    all_vis_md_texts += [vis_md_text]

print(vis_md_text)

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [05:17<00:00,  6.76s/it]
C:\Users\Ayrto\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


### Marginal Price Curve Estimation

The figure shown here is reproduced from the work in [this paper](https://ayrtonb.github.io/Merit-Order-Effect/#paper) by Ayrton Bourn which investigates the merit order effect of renewables in the GB and DE power markets in terms of the price and carbon reductions - the key findings are as follows:

* A LOWESS estimation of the non-linear marginal price curve for dispatchable generation shows high back-casting accuracy for Germany and Britain
* The evolving Merit Order Effect (MOE) was estimated through a time-adaptive model, enabling long-term trends to be captured
* In Britain the MOE has increased sharply since 2016, with a 0.67% price reduction per p.p. increase in RES penetration
* Disaggregation of the MOE by fuel-type highlights key differences in the transition paths of Britain and Germany

This figure will be updated on a weekly basis, the last update was at: 2021-07-03 13:45

<br>

#### Smoothed Price Time-Series Surface

This figure show

In [12]:
combined_md_text = '\n\n<br>\n\n'.join(['# Visualisations']+all_vis_md_texts)

print(combined_md_text.replace('vis/', '../docs/vis/'))

# Visualisations

<br>

### Measuring the Progress and Impacts of Decarbonising British Electricity

The figures shown here are attempts to replicate the visualisations from [this paper](https://www.sciencedirect.com/science/article/pii/S0301421516307017) by Dr Iain Staffell which finds that:

* CO2 emissions from British electricity have fallen 46% in the three years to June 2016.
* Emissions from imports and biomass are not attributed to electricity, but add 5%.
* Coal capacity fell 50% and output 75% due to prices, competition and legislation.
* Wind, solar and biomass provided 20% of demand in 2015, with a peak of 45%.
* Prices have become more volatile and net demand is falling towards must-run nuclear.

These figures will be updated on a weekly basis, the last update was at: 2021-07-03 13:40

<br>

#### Weekly Averaged Generation Mix

The following figure shows a stacked plot of the generation from different fuel types over time, averaged on a weekly basis. The original plot can 

In [14]:
with open('../docs/vis-contrib.md', 'w', encoding='utf-8') as f:
    f.write(combined_md_text)

In [ ]:
# need to write a func that will be run by the GH action 
# should check the time between now and any cron jobs descs, then run those that are within 24hrs
# no cron jobs will run more frequently than every 24hrs

# How to contribute?
# * Create a function that writes an image to the `docs/img/vis` directory 
# * Ensure that same function returns a markdown string which will render the desired text and images if loaded from the `docs` directory
# * Add the function name, schedule (in cron notation), and any kwargs to be passed to the function as a new item in the `data/vis_configs.json` file